<a href="https://colab.research.google.com/github/cjk5642/SteelersChatVolatility/blob/main/crawl_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [207]:
try:
  import psaw
except ModuleNotFoundError:
  !pip install psaw
  import psaw

import time
import requests
import requests.auth
from tqdm import tqdm
from datetime import datetime
from pytz import timezone
import pandas as pd
from praw.models import MoreComments
from psaw import PushshiftAPI

cred_path = "/content/drive/MyDrive/Data/Reddit/credentials.json"
with open(cred_path, 'r') as creds:
  creds = json.load(creds)

username = creds['USERNAME']
userAgent = "collectionbot/0.1 by " + username
clientId = creds['CLIENT_ID']
clientSecret = creds['SECRET_KEY']
password = creds['PASSWORD']

reddit = praw.Reddit(
    client_id=clientId,
    client_secret=clientSecret,
    password=password,
    user_agent=userAgent,
    username=username,
    check_for_async=False
)
api = PushshiftAPI(reddit)

INFO:psaw.PushshiftAPI:https://api.pushshift.io/meta


In [208]:
subreddit="steelers"
link_id = 'rzvzvq'
limit=100000

comments = api.search_comments(subreddit=subreddit, link_id = link_id, limit=limit)
res = list(comments)
print(f'Retrieved {len(res)} comments from Pushshift')

INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc


/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641764030
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641763946
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641763823
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641763687
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641763580
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641757232
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641756637
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641756446
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641756268
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?filter=id&filter=created_utc&subreddit=steelers&link_id=rzvzvq&limit=1000&metadata=true&sort=desc&before=1641756068
INFO:psaw.PushshiftAPI:https://api.pushshift.io/reddit/comment/search?

In [216]:
df = pd.DataFrame.from_dict([{"id": r.id, "datetime": str(datetime.fromtimestamp(r.created_utc)), "comment": r.body} for r in res])
df['datetime'] = df['datetime'].apply(pd.to_datetime)
df['datetime'] -= pd.to_timedelta("0 days 3 hours 0 seconds") # put into EST
df = df[(df['datetime'] >= "2022-01-09 13:00:00") & (df['datetime'] <= '2022-01-09 16:30:00')] # the time the game was player 1:00 PM to 4:30 PM EST
df = df[(df['comment'] != '[removed]') & (df['comment'] != '[deleted]')].reset_index(drop = True)

In [217]:
df.to_csv('final_steelers_game_2022.csv', index=False)